In [131]:

import numpy as np
import pandas as pd
import time

# ANSI codes for colors
class Color:
    RESET = "\033[0m"
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BLUE = "\033[94m"
    PURPLE = "\033[95m"
    CYAN = "\033[96m"
    WHITE = "\033[97m"

In [132]:
# function to receive equation input from users
# the constant is in the format "x_1, x_2, .....x_n"

def collect_data ():
  while True:
    try:
      row = int(input("Enter the number of Equations: \n (ensure that the number of equations is equal to number of variables)"))
      break
    except:
      print(Color.RED + "ERROR:Enter a valid number: " + Color.RESET)
  eqn = []
  for a in range(row):
    while True:
      try:
        while True:
          row_eqn = [float(x) for x in input(f"Enter Equation values for row {a+1} separated by space: ").split()]
          if len (row_eqn) == row:
            break
          else:
            print(Color.RED + f"Missing values present /n Enter {row} values for each row: " + Color.RESET)
        eqn.append(row_eqn)
        break
      except:
        print(Color.RED + "ERROR: Enter valid numbers separated by space: " + Color.RESET)
  while True:
    try:
      const = [float(y) for y in input(f"\n Enter Constant values separated by space: ").split()]
      break
    except:
      print(Color.RED + "ERROR: Enter valid numbers separated by space: " + Color.RESET)

  eqn = np.array(eqn)
  const = np.array(const)
  return eqn, const
# function to collect data from excel and csv files
def collect_df_data (file_path, sheet_name=None, header = None):
    """
    Read matrices from Excel and CSV files.

    Parameters:
    - file_path: The path to the file (Excel or CSV) containing matrices.
    - sheet_name: (Only for Excel) The name of the sheet to read. If None, reads the first sheet.

    Returns:
    - A list of matrices represented as NumPy arrays.
    """
    if file_path.endswith(".xlsx") or file_path.endswith(".xls"):
        # Read Excel file
        try:
            df = pd.read_excel(file_path, sheet_name=sheet_name, header=header)
        except Exception as e:
            print(f"Error reading Excel file: {e}")
            return None
    elif file_path.endswith(".csv"):
        # Read CSV file
        try:
            df = pd.read_csv(file_path, header=None)
        except Exception as e:
            print(f"Error reading CSV file: {e}")
            return None
    else:
        print("Unsupported file format. Supported formats: Excel (.xlsx, .xls), CSV (.csv)")
        return None

    # Convert DataFrame to NumPy array
    try:
        ar_eqn = [df.to_numpy()]
    except Exception as e:
        print(f"Error converting DataFrame to NumPy array: {e}")
        return None

    return ar_eqn
# function to collect the matrix data from a file
def collect_text_file ():
  file_name = input("Enter file name: \n include file path if different from python file: ")

  # reading the content of the text file and creating a matrix
  with open(filename, "r") as file:
    ar_eqn = [list(map(int, line.split())) for line in file.readlines()]
  # carry out gauss elimination
  return ar_eqn

In [147]:
# function to calculate the jacobian matrix of a function with  n variables
def n_jacobi (fn, n,  start, h = 1e-6):
  jacobi_mat = []
  func_mat = []
  for function in fn:
    func = function(*start)
    func_mat.append (func)
    partials = []
    for var in range(int(n)):
      perturbed_point = start.copy()
      perturbed_point[var] += h
      partial_dx = (function(*perturbed_point) - func)/h
      partials.append(partial_dx)
    jacobi_mat.append(partials)
  
  #print(np.shape(jacobi_mat))
  final_mat = np.array(func_mat).reshape((-1, 1))
  #return np.array(jacobi_mat), final_mat
  return jacobi_mat, final_mat
# using gauss-jordan method to solve the newton raphson equation on multivariant equations
def gauss_jordan_elimination (eqn, const):
  # augmenting the matrices
  aug_eqn = np.column_stack((eqn, const))

  # creating solution and iteration variable
  no_row, no_col = np.shape(aug_eqn)
  soln = np.zeros(no_row)


  # checking and pivoting elements where necessary
  for i in range(no_row):
    if aug_eqn[i][i] == 0:
      for k in range(i+1, no_col):
        if aug_eqn[k][i] != 0:
          temp = aug_eqn[i].copy()
          aug_eqn[i] = aug_eqn[k]
          aug_eqn[k] = temp
          break

    # carrying out gauss_jordan elimination
    aug_eqn[i] = aug_eqn[i]/aug_eqn[i][i]
    for j in range (no_row):
      if i != j:
        mul = aug_eqn[j][i]/aug_eqn[i][i]
        aug_eqn[j] = aug_eqn[j] - aug_eqn[i] * mul


    # getting the solution
  soln = aug_eqn[:, -1]

    # printing and returning solution
  #for s in range(no_row):
    #print("x_%d = %0.4f" %(s+1, soln[s]), end = "\t")
  return soln
def derivative(fn, x_0, h=1e-6):
    return (fn(x_0 + h) - fn(x_0))/h
def n_newton_raphson_gauss (fn, n, start, h = 1e-6, tol = 0.001, max_iter = 10):
    count = 0
    # creating a negative unity matrix
    #i_max = 
    #for i in range (n):
    while count < max_iter:
      var_values = []
      jacobi_mat, func_mat = n_jacobi (fn, n, start, h)
      func_soln = func_mat * (-1)
      delta_var = gauss_jordan_elimination (jacobi_mat, func_soln)
      
      var_values = []
      error_list = []
      for i in range (n):
        x_value = start[i] + delta_var[i]
        var_values.append(x_value)
        error = abs((x_value - start[i])/start[i])
        error_list.append(error)
    
      if sum(error_list) < tol:
        print (var_values)
        print(f"Number of iterations = {count}")
        break
      else:
        start = var_values.copy()
        count += 1
        if count == max_iter:
          print(Color.RED + "Newton Raphson Unsuccessful at Max iteration Value \n" + Color.RESET)
      
   
    return var_values

In [134]:
# defining a function that returns a matrix after carrying gauss_jordan elimination

def gauss_jordan_matrix (eqn, const):
  # augmenting the matrices
  aug_eqn = np.column_stack((eqn, const))

  # creating solution and iteration variable
  no_row, no_col = np.shape(aug_eqn)
  soln = np.zeros(no_row)


  # checking and pivoting elements where necessary
  for i in range(no_row):
    if aug_eqn[i][i] == 0:
      for k in range(i+1, no_col):
        if aug_eqn[k][i] != 0:
          temp = aug_eqn[i].copy()
          aug_eqn[i] = aug_eqn[k]
          aug_eqn[k] = temp
          break

    # carrying out gauss_jordan elimination
    aug_eqn[i] = aug_eqn[i]/aug_eqn[i][i]
    for j in range (no_row):
      if i != j:
        mul = aug_eqn[j][i]/aug_eqn[i][i]
        aug_eqn[j] = aug_eqn[j] - aug_eqn[i] * mul

  return aug_eqn
# defining the function to find the inverse of a matrix using gauss_jordan method
def gauss_jordan_inv (eqn):
  eqn = np.array(eqn)
  no_row, no_col = np.shape(eqn)

  # checking that the structure of the matrix is square and creating the identity and inverse matrix
  while True:
    try:
      if no_row == no_col:
        I = np.zeros((no_row, no_col))
        inv = np.zeros((no_row, no_col))
        break
      else:
        print("Enter a valid square matrix")
        sys.exit()
        break

    except:
      "check the input"

  for a in range(no_row):
    I[a, a] = 1

  # carrying out gauss_jordan elimination
  aug_eqn = gauss_jordan_matrix (eqn, I)

  # separating the inverse of the matrix from the augmented matrix
  for i in range(no_row):
    inv[i] = aug_eqn[i, no_row:]


  return inv



In [146]:
# program to carryout LU decomposition with pivoting on matrices

def LU_decomp_crout (a_eqn):
  no_row, no_col = np.shape(a_eqn)
  L = np.zeros((no_row, no_col))
  U = np.zeros((no_row, no_col))

  # carrying out pivoting
  p = np.zeros((no_row, no_col))
  for i in range (no_row):
    p[i, i] = 1

  for m in range(0, no_row-1):
    if a_eqn[m, m] == 0:
      for k in range(m+1, no_col):
        if a_eqn[k, m] != 0:
          m_temp = a_eqn[m].copy()
          a_eqn[m] = a_eqn[k]
          a_eqn[k] = m_temp
          p_temp = p[m].copy()
          p[m] = p[k]
          p[k] = p_temp
          break

  #getting the first row of the L column
  for i in range(no_row):
    L[i, 0] = a_eqn[i, 0]

  # putting the diagonal of U
  for j in range (no_row):
    U[j, j] = 1

  # calculating the first row of matrix U
  for a in range (1, no_col):
    U[0, a] = a_eqn[0, a] / L[0, 0]

  # calculating the other rows of matrix L and U
  for i in range (1, no_row):
    for j in range (1, i+1):
      L[i, j] = a_eqn[i, j] -np.sum(L[i, :j] * U[:j, j])

    for j in range (i + 1, no_row):
      U[i, j] = (a_eqn[i, j] - np.sum(L[i, :i] * U[:i, j]))/L[i, i]

  return L, U, p
# carrying forward substitution on the matrix
def forwardsub (L, p, const):
  no_row, no_col = np.shape(L)
  const = np.dot (p, const)
  soln = np.zeros(no_row)
  soln[0] = const[0]/L[0][0]

  for i in range (1, no_row):
    var = 0
    for j in range (0, i):
      var = var + L[i][j] * soln[j]
    soln[i] = (const[i] - var) / L[i][i]
  return soln
# carrying out backward substitution on the matrix
def backwardsub (U, L_soln):
  no_row, no_col = np.shape(U)
  soln = np.zeros(no_row)
  soln[-1] = L_soln[-1]

  for i in range (no_row-2, -1, -1):
    var = 0
    for j in range (i+1, no_row):
      var += U[i][j]*soln[j]
    soln[i] = L_soln[i] - var
  return soln
# carrying out the LU Decomposition procedure to find the solution to the matrix
def LU_decomp_crout_operation (eqn_test, const):
  L, U, p= LU_decomp_crout (eqn_test)
  L_soln = forwardsub (L, p, const)
  soln = backwardsub (U, L_soln)

  for a in range(len(soln)):
    print("x_%d = %0.4f" %(a+1, soln[a]))

  return soln
# program to find the inverse of the matrix using LU decomposition
def LU_matrix_inv (eqn):
  eqn = np.array(eqn)
  no_row, no_col = np.shape(eqn)

  # checking that the matrix is square and creating null matrices
  while True:
    try:
      if no_row == no_col:
        I = np.zeros ((no_row, no_col))
        inv = np.zeros((no_row, no_col))
        break
      else:
        print ("Enter a valid matrix with equal rows and columns")
        break
        sys.exit ()
    except:
      print ("check the inputs of the matrix")

  # creating an identity matrix
  for a in range (no_row):
    I[a, a] = 1

  # carryout LU decomposition with pivoting
  L, U, p = LU_decomp_crout (eqn)

  for i in range (no_row):
    I_col = I[:, i]
    L_soln = forwardsub (L, p, I_col)
    inv_row = backwardsub (U, L_soln)
    inv[:, i] = inv_row

  return inv
# function to calculate the jacobian matrix of a function with  n variables
def n_jacobi (fn, n,  start, h = 1e-6):
  jacobi_mat = []
  func_mat = []
  for function in fn:
    func = function(*start)
    func_mat.append (func)
    partials = []
    for var in range(int(n)):
      perturbed_point = start.copy()
      perturbed_point[var] += h
      partial_dx = (function(*perturbed_point) - func)/h
      partials.append(partial_dx)
    jacobi_mat.append(partials)
  
  #print(np.shape(jacobi_mat))
  final_mat = np.array(func_mat).reshape((-1, 1)) * (-1)
  func_mat = final_mat.tolist()
  #return np.array(jacobi_mat), final_mat
  return jacobi_mat, func_mat
# newton raphson for n variables using LU_decomposition 
def n_newton_raphson_LU(fn_ls, n, start, tol = 0.001, ite = 10, h=1e-6):
  # fn_ls should be in a list of functions
  # start is a list of starting values
  # n is the number of functions (variables)
  pass
  # counting the number of iterations
  count = 0
  
  while count < ite:
    var_values = []
    error_list = []
    jacobi_mat, func_mat = n_jacobi(fn_ls, n, start, h)
    jacobi_inv_mat = LU_matrix_inv (jacobi_mat)
    delta_var = jacobi_inv_mat.dot(func_mat)
    for i in range (n):
      x_value = start[i] + delta_var[i]
      [var_values.append(x) for x in x_value]
      error = abs((x_value - start[i])/start[i])
      error_list.append(error)
    
    if sum(error_list) < tol:
      print (var_values)
      print(f"Number of iterations = {count}")
      break
    else:
      start = var_values.copy()
      count += 1
  
  return var_values



In [150]:
fn = [
    lambda x, y: y - 0.5*(np.exp(x/2) + np.exp(-x/2)),
    lambda x, y: 25*(y**2) + 9*(x**2) -225
]

start = [2.5, 2]
gauss_start = time.time()
n_newton_raphson_gauss (fn, 2, start, tol=1e-6)
gauss_end = time.time()
gauss_time = gauss_end - gauss_start

LU_start = time.time()
n_newton_raphson_LU (fn, 2, start, tol=1e-6)
LU_end = time.time()
LU_time = LU_end - LU_start

print(LU_time - gauss_time)

[3.031155391720268, 2.3858656535626666]
Number of iterations = 3
0.004019737243652344


In [125]:
fn = [
    lambda x, y: -2 * (x**3) + 3*(y**2) + 42,
    lambda x, y: 5*(x**2) + 3*(y**3) - 69
]

start = [1, 1]
n_newton_raphson (fn, 2, start, tol=1e-6, ite =1)

[-5.6052584266268, 1.5614013586659188]

In [30]:
fn_list = [
    lambda x, y, z, a, b: x**2 + y**2 - 4,
    lambda x, y, z, a, b: x**2 + z**2 - 9,
    lambda x, y, z, a, b: y**2 + a**2 - 16,
    lambda x, y, z, a, b: z**2 + b**2 - 25,
    lambda x, y, z, a, b: a**2 + x**2 - 20
]

start_values = [1, 2, 0.7, 3, 1.2]
result = n_newton_raphson_gj (fn_list, 5, start=start_values, tol=1e-6, max_iter =100)
print(result)

[1.9999999999999996, 3.942573796936882e-08, 2.23606797749979, 4.0, 4.47213595499958]
Number of iterations = 50
[1.9999999999999996, 3.942573796936882e-08, 2.23606797749979, 4.0, 4.47213595499958]


In [39]:
fn = [lambda x, y: x**2 + y**2, lambda x, y: x**2 + y**3]
start = [1,3]
test = n_newton_raphson_gauss (fn , 2, start, max_iter=100)
print(test)


Newton Raphson Unsuccessful at Max iteration Value 

[0.5552940976525966, 0.9999999999999996]
